In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
topwords50 = ['지역', '산림', '문화', '숲', '사람', '교육', '자연', '마을', '이용', '산', '나무', '관광', '생활', '역사', '마음', '활용', '이야기', '미래', '가을', '예술', '청소년', '전통', '삶', '음식', '여름', '소나무', '봄', '이해', '여성', '겨울', '지식', '공감', '정신', '공동체', '사상', '성인', '공예', '정서', '창조', '고급', '남성', '신앙', '교양', '유람', '유람기', '향유', '의식주', '포용', '금송', '성황당', '서낭당', '친화력']
# '학습'을 제거하고 '교육'으로 통합시킴

In [3]:
data = pd.read_csv('숲문화_본문to형태소.csv')
print(data.shape, data.columns)
docs = data['morphs'].to_list()
print(len(docs))

(105874, 1) Index(['morphs'], dtype='object')
105874


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
for i in tqdm(range(len(docs))):
    filtered_lst = []
    for w in docs2[i].split():
        if w in topwords50:
            filtered_lst.append(w)
        elif w == '학습':
            filtered_lst.append('교육')
    else:
        docs2[i] = " ".join(filtered_lst)

  0%|          | 0/105874 [00:00<?, ?it/s]

In [ ]:
# #### 중복 제거 잘 됐나 개수로 확인
# print(len(docs[0]))
# print(len(docs2[0]))
# print(docs2[0][0:10])

In [5]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = docs2.copy()

In [6]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)
print(tdm.shape)

(105874, 50)


In [7]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.29837434, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.62721435, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.56546047, 0.2338028 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02767485, 0.        , 0.        , ..., 0.        , 0.        ,
        0.05586691],
       [0.        , 0.4019696 , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.298374,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.269361,0.0,0.000000,0.0,0.000000,0.000000,0.065647,0.320342,0.000000,0.794245,0.0,0.119638,0.000000,0.140164,0.000000,0.0,0.0,0.000000,0.123078,0.00000,0.148610,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.171671,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.000000,0.627214,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.429857,0.128365,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.120333,0.00000,0.581180,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.230428,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.565460,0.233803,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.512752,0.095700,0.046699,0.133549,0.308760,0.0,0.069763,0.000000,0.122598,0.039008,0.0,0.0,0.000000,0.017942,0.00000,0.303300,0.000000,0.000000,0.000000,0.0,0.284305,0.0,0.137432,0.000000,0.046608,0.000000,0.0,0.000000,0.150157,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.149288,0.035658,0.0,0.059235,0.0,0.000000,0.349215,0.017381,0.000000,0.121273,0.163554,0.0,0.000000,0.000000,0.000000,0.035423,0.0,0.0,0.000000,0.000000,0.06923,0.078692,0.205988,0.628784,0.036161,0.0,0.172114,0.0,0.436799,0.000000,0.296267,0.028037,0.0,0.061953,0.045451,0.197596,0.000000,0.0,0.048585,0.0,0.0,0.0,0.039868
4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.185062,0.000000,0.000000,0.000000,0.0,0.674538,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.173483,0.00000,0.000000,0.000000,0.352373,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.597066,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105869,0.000000,0.178967,0.0,0.194827,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.041610,0.0,0.000000,0.286192,0.024418,0.297888,0.056793,0.722161,0.0,0.222505,0.000000,0.052136,0.049766,0.0,0.0,0.000000,0.091561,0.00000,0.000000,0.144697,0.000000,0.000000,0.0,0.000000,0.0,0.087666,0.000000,0.237845,0.118170,0.0,0.000000,0.255421,0.069401,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
105870,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.551650,0.0,0.307629,0.135509,0.080932,0.000000,0.000000,0.217595,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.303472,0.00000,0.183212,0.000000,0.000000,0.336766,0.0,0.200361,0.0,0.000000,0.277783,0.394159,0.130555,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
105871,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.260216,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.243934,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.934229,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
105872,0.027675,0.000000,0.0,0.000000,0.0,0.046375,0.971926,0.000000,0.049968,0.0,0.041503,0.0,0.115722,0.000000,0.073066,0.000000,0.028323,0.032741,0.0,0.000000,0.069041,0.000000,0.000000,0.0,0.0,0.066718,0.045663,0.00000,0.000000,0.000000,0.046375,0.000000,0.0,0.000000,0.0,0.021860,0.020899,0.000000,0.078578,0.0,0.000000,0.031846,0.000000,0.042213,0.0,0.034041,0.0,0.0,0.0,0.055867


In [9]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [10]:
tfidf_DF

,가을,겨울,고급,공감,공동체,공예,관광,교양,교육,금송,나무,남성,마을,마음,문화,미래,봄,사람,사상,산,산림,삶,생활,서낭당,성인,소나무,숲,신앙,여름,여성,역사,예술,유람,음식,의식주,이야기,이용,이해,자연,전통,정서,정신,지식,지역,창조,청소년,친화력,포용,향유,활용
0,0.298374,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.269361,0.0,0.000000,0.0,0.000000,0.000000,0.065647,0.320342,0.000000,0.794245,0.0,0.119638,0.000000,0.140164,0.000000,0.0,0.0,0.000000,0.123078,0.00000,0.148610,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.171671,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.000000,0.627214,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.429857,0.128365,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.120333,0.00000,0.581180,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.230428,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.565460,0.233803,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.512752,0.095700,0.046699,0.133549,0.308760,0.0,0.069763,0.000000,0.122598,0.039008,0.0,0.0,0.000000,0.017942,0.00000,0.303300,0.000000,0.000000,0.000000,0.0,0.284305,0.0,0.137432,0.000000,0.046608,0.000000,0.0,0.000000,0.150157,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.149288,0.035658,0.0,0.059235,0.0,0.000000,0.349215,0.017381,0.000000,0.121273,0.163554,0.0,0.000000,0.000000,0.000000,0.035423,0.0,0.0,0.000000,0.000000,0.06923,0.078692,0.205988,0.628784,0.036161,0.0,0.172114,0.0,0.436799,0.000000,0.296267,0.028037,0.0,0.061953,0.045451,0.197596,0.000000,0.0,0.048585,0.0,0.0,0.0,0.039868
4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.185062,0.000000,0.000000,0.000000,0.0,0.674538,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.173483,0.00000,0.000000,0.000000,0.352373,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.597066,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105869,0.000000,0.178967,0.0,0.194827,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.041610,0.0,0.000000,0.286192,0.024418,0.297888,0.056793,0.722161,0.0,0.222505,0.000000,0.052136,0.049766,0.0,0.0,0.000000,0.091561,0.00000,0.000000,0.144697,0.000000,0.000000,0.0,0.000000,0.0,0.087666,0.000000,0.237845,0.118170,0.0,0.000000,0.255421,0.069401,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
105870,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.551650,0.0,0.307629,0.135509,0.080932,0.000000,0.000000,0.217595,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.303472,0.00000,0.183212,0.000000,0.000000,0.336766,0.0,0.200361,0.0,0.000000,0.277783,0.394159,0.130555,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
105871,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.260216,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.243934,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.934229,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
105872,0.027675,0.000000,0.0,0.000000,0.0,0.046375,0.971926,0.000000,0.049968,0.0,0.041503,0.0,0.115722,0.000000,0.073066,0.000000,0.028323,0.032741,0.0,0.000000,0.069041,0.000000,0.000000,0.0,0.0,0.066718,0.045663,0.00000,0.000000,0.000000,0.046375,0.000000,0.0,0.000000,0.0,0.021860,0.020899,0.000000,0.078578,0.0,0.000000,0.031846,0.000000,0.042213,0.0,0.034041,0.0,0.0,0.0,0.055867


In [ ]:
# 각 단어에 대한 TF-IDF의 합
word_TFIDF_sum = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_TFIDF_sum = word_TFIDF_sum.sort_values(by='tf-idf', ascending=False).reset_index(drop=True)
word_TFIDF_sum

In [12]:
# 각 단어에 대한 TF-IDF의 합
word_TFIDF_average = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.mean(axis=0).flat
})
word_TFIDF_average = word_TFIDF_average.sort_values(by='tf-idf', ascending=False).reset_index(drop=True)
word_TFIDF_average

,단어,tf-idf
0,사람,0.164257
1,숲,0.157451
2,문화,0.149831
3,마을,0.105019
4,자연,0.094819
5,지역,0.093314
6,나무,0.090809
7,이용,0.089242
8,마음,0.083240
9,산,0.082543


In [13]:
word_TFIDF_average.to_excel("숲문화_tfidf_top50.xlsx", index=False)